# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
!python3 -m pip install pandas-market-calendars yfinance
!python3 -m pip install matplotlib



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 37.0 MB/s eta 0:00:00:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=fbb9e246db941c2017f3cf08ac91d482552d6fdbc43598c123e4fac4ea28d4db
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-linux_x86_64.whl size=1049308 sha256=5a96b4236bf6a13c50b7b655d32827957aeaa6b958032764fc7b1782b49bff77
  Stored in directory: /home/codespace/.cache/pip/wheels/d1/df/a9/0202b051c65b11c992dd6db9f2babdd2c44ec7d35d511be5d3
Succ

In [2]:
import pandas as pd
import yfinance as yf                #yfinance as yf is used to download prices 

In [3]:
# YOUR CHANGES HERE

TICKERS = ["AAPL", "SPY", "TSLA", "SBUX", "MSFT", "NVDA"]  # includes 'S' for Sharma
RELEASE_DATE = "2025-09-01"                                 # I have done 2025-09-01 
N_MONTHS = 24                                                  # observations 

Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [4]:
#this code here is saying to compute a start date that is 24months plus 6 months which is like a cushion to ensure enough data
start = (pd.to_datetime(RELEASE_DATE) - pd.DateOffset(months=N_MONTHS + 6)).strftime("%Y-%m-%d")

# here we tell yfinance to return adjusted prices in the close field 
raw = yf.download(TICKERS, start=start, end=RELEASE_DATE, auto_adjust=True, progress=False)

#selects the closed block aka adjusted close 
close = raw["Close"].copy()
#ensured dates are sorted ascending 
close = close.sort_index()

#gets the actual last trading day of ach month 
last_dates = close.groupby(close.index.to_period("M")).apply(lambda df: df.index.max())
monthly = close.loc[last_dates.to_list()]

#keeps the last 24months and enforces column order 
monthly = monthly.tail(N_MONTHS)[TICKERS]

#format and save 
monthly.index = monthly.index.strftime("%Y-%m-%d")  # keep the real last-trading-day date
monthly.index.name = "date"
monthly.to_csv("historical_prices.tsv", sep="\t", index=True)





df = pd.read_csv("historical_prices.tsv", sep="\t")
assert list(df.columns) == ["date"] + TICKERS, "Header mismatch"
assert len(df) == N_MONTHS, f"Expected {N_MONTHS} rows, got {len(df)}"
print("  historical_prices.tsv")
print(df.head(3).to_string(index=False))

  historical_prices.tsv
      date       AAPL        SPY       TSLA      SBUX       MSFT      NVDA
2023-09-29 169.549286 417.865631 250.220001 86.798462 311.062317 43.475826
2023-10-31 169.113556 408.794342 200.839996 87.720947 333.090393 40.758278
2023-11-30 188.355316 446.135223 240.080002 94.953987 374.042236 46.745083


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [5]:
# YOUR CHANGES HERE

import pandas as pd

INPUT_FILE = "historical_prices.tsv"
OUTPUT_FILE = "historical_returns.tsv"

#reading the part 1 output 
prices = pd.read_csv(INPUT_FILE, sep="\t")


assert "date" in prices.columns, "Missing 'date' column in historical_prices.tsv"
ticker_cols = [c for c in prices.columns if c != "date"]
assert len(ticker_cols) == 6, f"Expected 6 tickers, found {len(ticker_cols)}: {ticker_cols}"

prices["date"] = pd.to_datetime(prices["date"])
prices = prices.sort_values("date").reset_index(drop=True)
numeric = prices[ticker_cols].astype(float)


returns = numeric.pct_change().dropna(how="any")         # drops the first (NaN) row
returns.index = prices.loc[returns.index, "date"]         # align dates to month-ends at t
returns.index.name = "date"




Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [6]:
# YOUR CHANGES HERE

out = returns.copy()
out.index = out.index.strftime("%Y-%m-%d")
out.to_csv(OUTPUT_FILE, sep="\t", index=True)

check = pd.read_csv(OUTPUT_FILE, sep="\t")
assert list(check.columns) == ["date"] + ticker_cols, "Header mismatch in historical_returns.tsv"
assert len(check) == 23, f"Expected 23 rows of returns, found {len(check)}"
assert check.drop(columns=["date"]).isna().sum().sum() == 0, "Found NaNs in returns"

print("  historical_returns.tsv")
display(check.head(3))
display(check.tail(3))


  historical_returns.tsv


,date,AAPL,SPY,TSLA,SBUX,MSFT,NVDA
0,2023-10-31,-0.002570,-0.021709,-0.197346,0.010628,0.070816,-0.062507
1,2023-11-30,0.113780,0.091344,0.195379,0.082455,0.122945,0.146886
2,2023-12-29,0.013583,0.045655,0.034988,-0.033132,-0.007574,0.058934


,date,AAPL,SPY,TSLA,SBUX,MSFT,NVDA
20,2025-06-30,0.021509,0.051386,-0.083126,0.091483,0.080481,0.169252
21,2025-07-31,0.011698,0.023032,-0.029560,-0.026956,0.072556,0.125831
22,2025-08-29,0.119639,0.020520,0.083044,-0.004353,-0.048692,-0.020746


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [7]:
# YOUR CHANGES HERE

import pandas as pd

INPUT_FILE = "historical_returns.tsv"
OUTPUT_FILE = "estimated_returns.tsv"


# Load returns
rets = pd.read_csv(INPUT_FILE, sep="\t")
assert "date" in rets.columns, "Missing 'date' column in historical_returns.tsv"
tickers = [c for c in rets.columns if c != "date"]
assert len(tickers) == 6, f"Expected 6 tickers, found {len(tickers)}"

R = rets[tickers].astype(float)

mu = R.mean(axis=0)

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [8]:
# YOUR CHANGES HERE

out = mu.reindex(tickers).reset_index()
out.columns = ["asset", "estimated_return"]
out.to_csv(OUTPUT_FILE, sep="\t", index=False)

check = pd.read_csv(OUTPUT_FILE, sep="\t")
assert list(check.columns) == ["asset", "estimated_return"]
assert len(check) == 6
assert check["estimated_return"].isna().sum() == 0

print(" estimated_returns.tsv")


 estimated_returns.tsv


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [9]:
# YOUR CHANGES HERE

import pandas as pd

INPUT = "historical_returns.tsv"
OUTPUT = "estimated_covariance.tsv"


rets = pd.read_csv(INPUT, sep="\t")
assert "date" in rets.columns, "Missing 'date' column in historical_returns.tsv"
tickers = [c for c in rets.columns if c != "date"]

R = rets[tickers].astype(float)
Sigma = R.cov()


Sigma = Sigma.loc[tickers, tickers]
Sigma.index.name = ""               # makes the first header cell blank
Sigma.to_csv(OUTPUT, sep="\t")      # in pandas matrix format 





Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [10]:
# YOUR CHANGES HERE

with open(OUTPUT, "r", encoding="utf-8") as f:
    header = f.readline().rstrip("\n").split("\t")
assert header[0] == "" and header[1:] == tickers, "Header format not as required"
chk = pd.read_csv(OUTPUT, sep="\t", index_col=0)
assert list(chk.columns) == tickers and list(chk.index) == tickers and chk.shape == (6, 6)

print("  estimated_covariance.tsv")

  estimated_covariance.tsv


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [11]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

INPUT = "estimated_returns.tsv"
OUTPUT = "maximum_return.tsv"

er = pd.read_csv(INPUT, sep="\t")
assert list(er.columns) == ["asset", "estimated_return"], "estimated_returns.tsv header must be ['asset','estimated_return']"
assert len(er) == 6, "Expected 6 assets"

imax = er["estimated_return"].idxmax()
assets = er["asset"].tolist()

alloc = np.zeros(len(assets), dtype=float)
alloc[imax] = 1.0

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [12]:
# YOUR CHANGES HERE

out = pd.DataFrame({"asset": assets, "allocation": alloc})
out.to_csv(OUTPUT, sep="\t", index=False)

check = pd.read_csv(OUTPUT, sep="\t")
assert list(check.columns) == ["asset", "allocation"]
assert abs(check["allocation"].sum() - 1.0) < 1e-9
assert (check["allocation"] >= 0).all()
print(" maximum_return.tsv")
check

 maximum_return.tsv


,asset,allocation
0,AAPL,0.0
1,SPY,0.0
2,TSLA,0.0
3,SBUX,0.0
4,MSFT,0.0
5,NVDA,1.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [13]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

from scipy.optimize import minimize

INPUT = "estimated_covariance.tsv"
OUTPUT = "minimum_risk.tsv"

Sigma_df = pd.read_csv(INPUT, sep="\t", index_col=0)
tickers = list(Sigma_df.columns)
Sigma = Sigma_df.loc[tickers, tickers].to_numpy(dtype=float)
n = len(tickers)

def var_objective(w, Sigma=Sigma):
    return float(w @ Sigma @ w)

cons = ({"type": "eq", "fun": lambda w: np.sum(w) - 1.0},)
bounds = [(0.0, 1.0)] * n
x0 = np.full(n, 1.0 / n)

res = minimize(var_objective, x0, method="SLSQP", bounds=bounds, constraints=cons)
if not res.success:
    raise RuntimeError("Min-variance optimization failed: " + res.message)

w = res.x

w[w < 0] = 0.0
s = w.sum()
if s <= 0:
    raise ValueError("Degenerate solution: sum of weights <= 0")
w = w / s



Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [14]:
# YOUR CHANGES HERE

out = pd.DataFrame({"asset": tickers, "allocation": w})
out.to_csv(OUTPUT, sep="\t", index=False)

check = pd.read_csv(OUTPUT, sep="\t")
assert list(check.columns) == ["asset", "allocation"]
assert abs(check["allocation"].sum() - 1.0) < 1e-8
assert (check["allocation"] >= -1e-10).all()  # allows tiny numerical noise

print(" minimum_risk.tsv")
check

 minimum_risk.tsv


,asset,allocation
0,AAPL,3.396998e-01
1,SPY,3.663415e-01
2,TSLA,1.785062e-17
3,SBUX,6.290436e-02
4,MSFT,2.310543e-01
5,NVDA,6.107303e-18


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [15]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Inputs from previous parts
ER_FILE   = "estimated_returns.tsv"
COV_FILE  = "estimated_covariance.tsv"
MIN_FILE  = "minimum_risk.tsv"
MAX_FILE  = "maximum_return.tsv"
OUT_FILE  = "efficient_frontier.tsv"

# Loading inputs and align tickers consistently
Sigma_df = pd.read_csv(COV_FILE, sep="\t", index_col=0)
tickers = list(Sigma_df.columns)                     # canonical order
Sigma = Sigma_df.loc[tickers, tickers].to_numpy(float)

er = pd.read_csv(ER_FILE, sep="\t")
mu_map = dict(zip(er["asset"], er["estimated_return"]))
mu = np.array([mu_map[t] for t in tickers], dtype=float)

w_min_df = pd.read_csv(MIN_FILE, sep="\t")
w_min_map = dict(zip(w_min_df["asset"], w_min_df["allocation"]))
w_min = np.array([w_min_map[t] for t in tickers], dtype=float)

w_max_df = pd.read_csv(MAX_FILE, sep="\t")
w_max_map = dict(zip(w_max_df["asset"], w_max_df["allocation"]))
w_max = np.array([w_max_map[t] for t in tickers], dtype=float)

# weights sum to 1 and are >=0
assert abs(w_min.sum() - 1) < 1e-8 and (w_min >= -1e-12).all()
assert abs(w_max.sum() - 1) < 1e-8 and (w_max >= -1e-12).all()

# The Helper functions 
def port_stats(w, mu, Sigma):
    r = float(w @ mu)
    v = float(w @ Sigma @ w)
    s = np.sqrt(v)
    return r, v, s

def solve_min_var_at_return(Sigma, mu, target, w0=None):
    """
    Minimize w^T Σ w subject to:
      1) sum(w) = 1
      2) mu^T w = target
      3) 0 <= w <= 1  (long-only)
    """
    n = len(mu)
    bounds = [(0.0, 1.0)] * n
    cons = (
        {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type": "eq", "fun": lambda w, mu=mu, target=target: float(w @ mu) - target},
    )
    if w0 is None:
        w0 = np.full(n, 1.0 / n)
    res = minimize(lambda w: w @ Sigma @ w, w0, method="SLSQP", bounds=bounds, constraints=cons)
    return res

# endpoint returns
r_min, _, s_min = port_stats(w_min, mu, Sigma)
r_max, _, s_max = port_stats(w_max, mu, Sigma)

#  101 evenly spaced returns 
targets = np.linspace(r_min, r_max, 101)

rows = []
for i, tr in enumerate(targets):
    if i == 0:
        w = w_min.copy()
    elif i == 100:
        w = w_max.copy()
    else:
        # convex mix of endpoints, with alpha matching target return
        alpha = (tr - r_min) / (r_max - r_min) if r_max > r_min else 0.0
        w0 = (1 - alpha) * w_min + alpha * w_max
        res = solve_min_var_at_return(Sigma, mu, tr, w0=w0)
        if not res.success:
            # Then Fallback to convex blend 
            w = np.clip(w0, 0, 1)
            w = w / w.sum()
        else:
            w = res.x
            # Clean tiny numerical noise and renormalize
            w[w < 0] = 0.0
            w = w / w.sum()

    r, v, s = port_stats(w, mu, Sigma)
    row = {"index": i, "return": r, "risk": s}
    row.update({tickers[j]: w[j] for j in range(len(tickers))})
    rows.append(row)




Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [16]:
# YOUR CHANGES HERE

frontier = pd.DataFrame(rows, columns=["index", "return", "risk"] + tickers)
frontier.to_csv(OUT_FILE, sep="\t", index=False)

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["index", "return", "risk"] + tickers
assert len(chk) == 101
assert np.allclose(chk.loc[0, tickers].to_numpy(float), w_min, atol=1e-6)   
assert np.allclose(chk.loc[100, tickers].to_numpy(float), w_max, atol=1e-6) 
print("  efficient_frontier.tsv with 101 portfolios.")

  efficient_frontier.tsv with 101 portfolios.


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [17]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

ER_FILE   = "estimated_returns.tsv"
COV_FILE  = "estimated_covariance.tsv"
RETS_FILE = "historical_returns.tsv"
OUT_FILE  = "max_return_probabilities.tsv"

er = pd.read_csv(ER_FILE, sep="\t")                      # columns: asset, estimated_return
Sigma_df = pd.read_csv(COV_FILE, sep="\t", index_col=0)  # covariance matrix
rets = pd.read_csv(RETS_FILE, sep="\t")                  

tickers = list(Sigma_df.columns)                         # canonical order
mu_map = dict(zip(er["asset"], er["estimated_return"]))
mu = np.array([mu_map[t] for t in tickers], dtype=float)
Sigma = Sigma_df.loc[tickers, tickers].to_numpy(dtype=float)

n_assets = len(tickers)
n_samples = len(rets)    

min_eig = np.linalg.eigvalsh((Sigma + Sigma.T) / 2).min()
if min_eig < 0:
    Sigma = Sigma + np.eye(n_assets) * (-(min_eig) + 1e-12)

# Resampling loop 
rng = np.random.default_rng(42)
counts = np.zeros(n_assets, dtype=int)
num_trials = 1000

for _ in range(num_trials):
    samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples, check_valid="ignore")
    mu_hat = samples.mean(axis=0)               
    top_idx = int(np.argmax(mu_hat))            
    counts[top_idx] += 1

probs = counts / num_trials




Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [18]:
# YOUR CHANGES HERE

out = pd.DataFrame({"asset": tickers, "probability": probs})
out.to_csv(OUT_FILE, sep="\t", index=False)

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["asset", "probability"]
assert np.isclose(chk["probability"].sum(), 1.0), "Probabilities should sum to ~1"
print("  max_return_probabilities.tsv")
chk

  max_return_probabilities.tsv


,asset,probability
0,AAPL,0.004
1,SPY,0.001
2,TSLA,0.153
3,SBUX,0.006
4,MSFT,0.002
5,NVDA,0.834


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [19]:
content = """I acknowledge the use of the following libraries and resources:

- pandas, numpy, matplotlib — data manipulation and plotting
- yfinance — downloading market data
- scipy.optimize — portfolio optimization

I discussed this assignment with no one and did not use any additional sources beyond the course materials,
other than occasional Google searches to help understand concepts covered in class. 
"""

with open("acknowledgements.txt", "w", encoding="utf-8") as f:
    f.write(content)

import os
print("exists:", os.path.exists("acknowledgements.txt"))


exists: True


I acknowledge the use of the following libraries and resources:

- pandas, numpy, matplotlib — data manipulation and plotting
- yfinance — downloading market data
- scipy.optimize — portfolio optimization 

I discussed this assignment with no one and did not use any additional sources beyond the course materials other then google here and there to clarify materials learned in class to make sure I understand what im doing. 

Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.